# Import

In [1]:
import pandas as pd

# Data Load & Pre-processing

In [14]:
train = pd.read_csv('../../data/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('../../data/test.csv', encoding = 'utf-8-sig')

train['공종'].unique()

array(['건축 > 철근콘크리트공사', '토목 > 터널공사', '토목 > 하천공사', '건축 > 해체 및 철거공사',
       '건축 > 타일 및 돌공사', '건축 > 철골공사', '건축 > 금속공사', '토목 > 해체 및 철거공사',
       '기타 > 기타', '건축 > 수장공사', '건축 > 미장공사', '토목 > 가설공사', '건축 > 조적공사',
       '토목 > 토공사', '건축 > 도장공사', '건축 > 건축 토공사', '토목 > 관공사 부대공사',
       '건축 > 가설공사', '기계설비 > 기계설비공사', '건축 > 목공사', '토목 > 관공사', '토목 > 말뚝공사',
       '건축 > 지붕 및 홈통공사', '토목 > 항만공사', '토목 > 교량공사', '기계설비 > 가설공사',
       '건축 > 건축물 부대공사', '전기설비 > 전기설비공사', '건축 > 창호 및 유리공사',
       '토목 > 철근콘크리트공사', '토목 > 도로 및 포장공사', '산업설비 > 가설공사', '토목 > 철도 및 궤도공사',
       '건축 > 방수공사', '토목 > 강구조물공사', '건축 > 지정공사', '산업설비 > 산업설비공사',
       '토목 > 지반개량공사', '토목 > 댐 및 제방공사', '토목 > 지반조사', '건축 > 조경공사',
       '토목 > 프리캐스트 콘크리트공사', '전기설비 > 가설공사', '기계설비 > 해체 및 철거공사',
       '전기설비 > 해체 및 철거공사', '건축 > 특수 건축물공사', nan, '통신설비 > 통신설비공사',
       '건축 > 지반조사', '산업설비 > 해체 및 철거공사'], dtype=object)

In [17]:
# '공종' 컬럼의 결측값을 채우기 위한 매핑 (">" 포함)
construction_fill_values = {
    "TRAIN_02856": "건축 > 마감공사",           # 건축 / 건축물 / 공동주택, 작업프로세스: 기타
    "TRAIN_04706": "건축 > 철근콘크리트공사",     # 건축 / 건축물 / 운동시설, 사고객체: 건물
    "TRAIN_06235": "건축 > 마감공사",           # 건축 / 건축물 / 공동주택, 작업프로세스: 청소작업
    "TRAIN_09122": "기타 > 기타공사",           # 공사종류 정보 부족 → 기타공사
    "TRAIN_13429": "건축 > 설비공사",           # 건축 / 건축물 / 공장, 사고객체: 기타
    "TRAIN_13708": "토목 > 토공사",            # 건축 / 건축물 / 기타, 사고객체: 덤프트럭
    "TRAIN_13866": "조경 > 조경공사",           # 조경 / 기타, 작업프로세스: 운반작업
    "TRAIN_14143": "토목 > 부지조성공사",        # 토목 / 기타 / 부지조성, 장소: 부지조성
    "TRAIN_14715": "조경 > 조경공사",           # 건축 / 건축물 / 관광 휴게시설, 작업프로세스: 이동
    "TRAIN_15805": "건축 > 마감공사",           # 건축 / 건축물 / 공동주택, 작업프로세스: 운반작업
    "TRAIN_18108": "기타 > 기타공사"            # 건축 / 건축물 / 기타
}

for k in construction_fill_values:
    print(construction_fill_values[k] in train['공종'].unique())

# 각 ID에 해당하는 '공종' 컬럼 값을 채우기
for record_id, construction_type in construction_fill_values.items():
    train.loc[train["ID"] == record_id, "공종"] = construction_type

# 결측치 확인 (0이 나오면 결측치 처리 완료)
missing_values_after_filling = train["공종"].isnull().sum()
print(f"'공종' 컬럼의 남은 결측치 개수: {missing_values_after_filling}")



True
True
True
True
True
True
True
True
True
True
True
'공종' 컬럼의 남은 결측치 개수: 0


In [7]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [8]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [10]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))
combined_test_data

,question
0,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
1,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
2,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
3,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
4,"공사종류 대분류 '토목', 중분류 '교량' 공사 중 공종 대분류 '토목', 중분류 ..."
...,...
959,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
960,"공사종류 대분류 '토목', 중분류 '철도' 공사 중 공종 대분류 '토목', 중분류 ..."
961,"공사종류 대분류 '토목', 중분류 '상하수도' 공사 중 공종 대분류 '토목', 중분..."
962,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류..."
